In [1]:
! pip install pyspark

     |████████████████████████████████| 215.6MB 78kB/s 
     |████████████████████████████████| 204kB 38.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215965824 sha256=d004e216e1550b50d00a3aef8a5e06c94b8a680cb1bb1dffd5cef96ba2140f0b
  Stored in directory: /tmp/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/gender_submission.csv
/kaggle/input/test.csv
/kaggle/input/train.csv


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

In [4]:
sdf_train = spark.read.csv("../input/train.csv",inferSchema=True,header=True)
print(sdf_train.printSchema())
pdf = sdf_train.limit(10).toPandas()
pdf.T

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

None


,0,1,2,3,4,5,6,7,8,9
PassengerId,1,2,3,4,5,6,7,8,9,10
Survived,0,1,1,1,0,0,0,0,1,1
Pclass,3,1,3,1,3,3,1,3,3,2
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry","Moran, Mr. James","McCarthy, Mr. Timothy J","Palsson, Master. Gosta Leonard","Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)","Nasser, Mrs. Nicholas (Adele Achem)"
Sex,male,female,female,female,male,male,male,male,female,female
Age,22,38,26,35,35,NaN,54,2,27,14
SibSp,1,1,0,1,0,0,0,3,0,1
Parch,0,0,0,0,0,0,0,1,2,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450,330877,17463,349909,347742,237736
Fare,7.25,71.2833,7.925,53.1,8.05,8.4583,51.8625,21.075,11.1333,30.0708


In [5]:
sdf_test = spark.read.csv("../input/test.csv",inferSchema=True,header=True)
# sdf_train.printSchema()
pdf = sdf_test.limit(10).toPandas()

pdf.T

,0,1,2,3,4,5,6,7,8,9
PassengerId,892,893,894,895,896,897,898,899,900,901
Pclass,3,3,2,3,3,3,3,2,3,3
Name,"Kelly, Mr. James","Wilkes, Mrs. James (Ellen Needs)","Myles, Mr. Thomas Francis","Wirz, Mr. Albert","Hirvonen, Mrs. Alexander (Helga E Lindqvist)","Svensson, Mr. Johan Cervin","Connolly, Miss. Kate","Caldwell, Mr. Albert Francis","Abrahim, Mrs. Joseph (Sophie Halaut Easu)","Davies, Mr. John Samuel"
Sex,male,female,male,male,female,male,female,male,female,male
Age,34.5,47,62,27,22,14,30,26,18,21
SibSp,0,1,0,0,1,0,0,1,0,2
Parch,0,0,0,0,1,0,0,1,0,0
Ticket,330911,363272,240276,315154,3101298,7538,330972,248738,2657,A/4 48871
Fare,7.8292,7,9.6875,8.6625,12.2875,9.225,7.6292,29,7.2292,24.15
Cabin,None,None,None,None,None,None,None,None,None,None


In [6]:
numeric_cols = ['PassengerId','Survived', 'Pclass','Age', 'SibSp','Parch','Ticket','Fare'] 
numeric_features = ['Pclass','Age', 'SibSp','Parch','Fare'] 


In [7]:
from pyspark.sql import DataFrame 
from pyspark.sql import functions as F
from pyspark.ml.feature import Imputer

# default is mean
def _claenup(sdf: DataFrame,colList: list):
    imputer = Imputer(inputCols = colList,
                     outputCols = colList)
    sdf = imputer.fit(sdf).transform(sdf)
    return sdf

sdf_train_cleaned = _claenup(sdf_train,['Fare','Age'])
sdf_train_cleaned.limit(5).toPandas().T


,0,1,2,3,4
PassengerId,1,2,3,4,5
Survived,0,1,1,1,0
Pclass,3,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry"
Sex,male,female,female,female,male
Age,22,38,26,35,35
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450
Fare,7.25,71.2833,7.925,53.1,8.05


In [8]:
# sdf_train_subset.select(numeric_features).printSchema() #.toPandas().T

In [9]:
_stages = []

In [10]:
# from pyspark.sql import functions as F
# sdf_train_cleaned.groupBy("Sex").agg(F.count(sdf_train_cleaned["Sex"])).show()
# sdf_train_cleaned.groupBy("Embarked").agg(F.count(sdf_train_cleaned["Embarked"])).show()

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

string_features = [ 'Embarked', 'Sex'] # 'Cabin', 

def _strIndexer(_stages):
    strIndexer = [StringIndexer(inputCol=column, outputCol=column+"_Index", handleInvalid = "skip") for column in string_features ] #.fit(sdf_train_subset)
    _stages += strIndexer

In [12]:
def _oneHotEnc(_stages):
    oneHotEnc = [OneHotEncoderEstimator(inputCols= [column +"_Index"  for column in string_features]  , outputCols= [column+"_Enc" for column in string_features] )]
    _stages += oneHotEnc


In [13]:
from pyspark.ml.feature import VectorAssembler
# assemblerInput.clear()
def _vectAssembler(_stages):
    assemblerInput =  [f  for f in numeric_features]  
    assemblerInput += [f + "_Enc" for f in string_features]
    print(assemblerInput) 
    # assemblerInput.append("sexIndex")
    vectAssembler = VectorAssembler(inputCols  = assemblerInput, outputCol = "vect_features") #.fit(sdf_train_subset)  
    _stages += [vectAssembler]


In [14]:
from pyspark.ml.feature import PCA
# help(PCA)
def _pac(_stages):
    pca = PCA(inputCol = "vect_features", outputCol = "pca_features",k = 5)
    _stages += [pca]

In [15]:
from pyspark.ml.classification import RandomForestClassifier
# help(RandomForestClassifier)
def _rf(_stages):
    rf = RandomForestClassifier(labelCol = 'Survived', featuresCol = 'pca_features', numTrees = 100, maxDepth = 4, maxBins = 1000)
    _stages += [rf]

In [16]:
# _stages += [oneHotEnc, vectAssembler,rf]
_stages = []
_strIndexer(_stages)
_oneHotEnc(_stages)
_vectAssembler(_stages)
_pac(_stages)
_rf(_stages)
_stages

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Enc', 'Sex_Enc']


[StringIndexer_56a16de9fdd2,
 StringIndexer_4e1c67b47f28,
 OneHotEncoderEstimator_d170bff8037e,
 VectorAssembler_05c997e263f2,
 PCA_f55265094d83,
 RandomForestClassifier_1d5410e0b197]

In [17]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = _stages)

In [18]:
model = pipeline.fit(sdf_train_cleaned)

In [19]:
numeric_cols_test = ['PassengerId', 'Pclass','Age', 'SibSp','Parch','Ticket','Fare', 'Sex'] 

sdf_test_subset = sdf_test.withColumn('Ticket', sdf_test['Ticket'].cast("double"))

sdf_test_subset = _claenup(sdf_test_subset,['Fare','Age'])

In [20]:
sdf_predict = model.transform(sdf_test_subset)

In [21]:
sdf_predict.toPandas().T

,0,1,2,3,4,5,6,7,8,9,...,408,409,410,411,412,413,414,415,416,417
PassengerId,892,893,894,895,896,897,898,899,900,901,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
Pclass,3,3,2,3,3,3,3,2,3,3,...,3,3,3,1,3,3,1,3,3,3
Name,"Kelly, Mr. James","Wilkes, Mrs. James (Ellen Needs)","Myles, Mr. Thomas Francis","Wirz, Mr. Albert","Hirvonen, Mrs. Alexander (Helga E Lindqvist)","Svensson, Mr. Johan Cervin","Connolly, Miss. Kate","Caldwell, Mr. Albert Francis","Abrahim, Mrs. Joseph (Sophie Halaut Easu)","Davies, Mr. John Samuel",...,"Riordan, Miss. Johanna Hannah""""""""","Peacock, Miss. Treasteall","Naughton, Miss. Hannah","Minahan, Mrs. William Edward (Lillian E Thorpe)","Henriksson, Miss. Jenny Lovisa","Spector, Mr. Woolf","Oliva y Ocana, Dona. Fermina","Saether, Mr. Simon Sivertsen","Ware, Mr. Frederick","Peter, Master. Michael J"
Sex,male,female,male,male,female,male,female,male,female,male,...,female,female,female,female,female,male,female,male,male,male
Age,34.5,47,62,27,22,14,30,26,18,21,...,30.2726,3,30.2726,37,28,30.2726,39,38.5,30.2726,30.2726
SibSp,0,1,0,0,1,0,0,1,0,2,...,0,1,0,1,0,0,0,0,0,1
Parch,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
Ticket,330911,363272,240276,315154,3.1013e+06,7538,330972,248738,2657,NaN,...,334915,NaN,365237,19928,347086,NaN,NaN,NaN,359309,2668
Fare,7.8292,7,9.6875,8.6625,12.2875,9.225,7.6292,29,7.2292,24.15,...,7.7208,13.775,7.75,90,7.775,8.05,108.9,7.25,8.05,22.3583
Cabin,None,None,None,None,None,None,None,None,None,None,...,None,None,None,C78,None,None,C105,None,None,None


In [22]:
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator 

# sdf_train_sample = sdf_train_cleaned.sample(False,0.1).cache()
# # sdf_test_sample.count()
# pca_model = PCA(inputCol = "vect_features", outputCol = "pca_features",k = 5)
# cv_rf = RandomForestClassifier(labelCol = "Survived", featuresCol = "pca_features")
# cv_pipeline = Pipeline(stages = [pca_model,rf])

# param_grid = ParamGridBuilder().addGrid(pca_model.k, [10, 20, 30, 40, 50])\
#                                 .addGrid(cv_rf.numTrees, [20, 30, 50])\
#                                 .build()
# cross_val = CrossValidator(estimator = cv_pipeline,
#                           estimatorParamMaps = param_grid,
#                           evaluator = BinaryClassificationEvaluator(labelCol= 'STP_UP_IND',
#                                                                    rawPredictionCol = 'probability',
#                                                                    metricName = 'areaUnderROC'),
#                           numFolds = 3)
# cv_model = cross_val.fit(sdf_test_sample)

# cv_prediction = cv_model.transform(sdf_test_subset)

# evaluator= BinaryClassificationEvaluator(labelCol = "STP_UP_IND", rawPredictionCol="probability", metricName= "areaUnderROC")
# accuracy = evaluator.evaluate(predictions)
# accuracy
# # eval = BinaryClassificationEvaluator()
evaluator = BinaryClassificationEvaluator(labelCol = 'prediction')
evaluator.evaluate(sdf_predict)

1.0

In [23]:
sdf_submission = sdf_predict.select('PassengerId','prediction').withColumn('Survived',sdf_predict['prediction'].cast('integer')).select('PassengerId','Survived')
sdf_submission.toPandas().T

,0,1,2,3,4,5,6,7,8,9,...,408,409,410,411,412,413,414,415,416,417
PassengerId,892,893,894,895,896,897,898,899,900,901,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
Survived,0,0,0,0,0,0,0,1,1,0,...,0,1,0,1,1,0,1,0,0,0


In [24]:
sdf_submission.coalesce(1).write.csv("submission",mode="overwrite",header=True)


In [25]:
print(os.listdir('submission'))


['part-00000-744c414b-c990-4d00-bdd1-61cd130d92d3-c000.csv', '.part-00000-744c414b-c990-4d00-bdd1-61cd130d92d3-c000.csv.crc', '_SUCCESS', '._SUCCESS.crc']


<a href="submission/part-00000-6bf2b4f5-566e-474a-adc4-7981ac0609ea-c000.csv">Download Submission</a>

### Please note this is a INPROGRESS notebook!